# Top

In [1]:
_LCM = lambda f, g: f.parent().monomial_lcm(f, g)
_LC = lambda f : f.lc()
_LM = lambda f : f.lm()
_LT = lambda f : f.lt()

In [2]:
def inter_reduction(Q):

    if not Q:
        return Q  # if Q is empty we cannot get a base ring
    base_ring = next(iter(Q)).base_ring()

    Q = set(Q)
    while True:
        Qbar = set(Q)
        for p in sorted(Qbar):
            Q.remove(p)
            h = p.reduce(Q)
            if not h.is_zero():
                Q.add(h)
        if Qbar == Q:
            if base_ring.is_field():
                return set(f.lc()**(-1) * f for f in Qbar)
            else:
                return Qbar

# Remake Algoritma F4

In [3]:
def cxa(fi,fj):
    return lcm(fi.lc(), fj.lc()) * _LCM(fi, fj)

def S(P):
    return min(P)

LTS = lambda G : set(_LT(g) for g in G)

def F4_remake(Id):
    
    G = set(Id.gens())
    
    for g in G:
        if not _LC(g).is_unit():
            return _LC(g)
    
    G = inter_reduction(G)
        
    P = set((f,g) for f in G for g in G if f != g if f < g)
    
    H = set()

    while P:
        
        for p in P:
            for i in p:
                if not _LC(i).is_unit():
                    return _LC(i)
        
        fi, fj = S(P)
        
        
        H = H.union({cxa(fi, fj)*fj//_LM(fj)})
        H = H.union({cxa(fi, fj)*fi//_LM(fi)})
        
        P.remove((fi, fj))
        
        H = H.union(G)
        
        H = inter_reduction(H)
        
        P = P.union(set((f,h) for f in G for h in H if _LT(h) not in LTS(G))) #cek lagi
        
        G = H

    return Sequence(G).reduced()

# ! Proposisi 16 : Case 2 only

In [4]:
def factorize(n):
    factor_n = factor(n)
    x, y = factor_n[0]
    q = n/(x^y)
    return (x^y), q

# ! Corolary 14

In [5]:
def fki(g1, g2, u, v, a, b, R):
    idtmp1 = R.ideal(g1)
    idtmp2 = R.ideal(g2)
    rg1 = Sequence(idtmp1)    
    rg2 = Sequence(idtmp2)
    rg1.append(a)
    rg2.append(b)
    G = set()
    for k in rg1:
        for l in rg2:
            f = (u*a*_LCM(_LM(k),_LM(l))*_LC(k)*l//_LM(l)) + (v*b*_LCM(_LM(k),_LM(l))*_LC(l)*k//_LM(k))
            G = G.union({f})
    return Sequence(G).reduced()

# ! Algoritma Eder Hofmann ft. F4

In [6]:
def GB_PIR_F4(I):
    R = I.ring()
    Rn = R.base_ring()
    
    if Rn.is_field():
        
        return I.groebner_basis('macaulay2:f4')
    else:

        factor_n = factor(Rn.cardinality())
        if len(factor_n) == 1:
            
            
            return I.groebner_basis('singular').reduced()
        else:
            
            test = F4_remake(I)
            if isinstance(test, list):
                
                return test
                
            else:
                p, q = factorize(Rn.cardinality())
                Rp = I.ring().change_ring(Integers(p))
                Ip = Rp.ideal(Sequence(Rp.ideal(I).gens()).reduced())
                Gbp = GB_PIR_F4(Ip)
                Rq = I.ring().change_ring(Integers(q))
                Iq = Rq.ideal(Sequence(Rq.ideal(I).gens()).reduced())
                Gbq = GB_PIR_F4(Iq)
                
                d, u, v = xgcd(p,q)

                G = fki(Gbp, Gbq, R(u),R(v),R(p),R(q),R)

                return G.reduced()
        
        
        
#

In [8]:
R.<a,b,c,d,e,f,g,h,i> = PolynomialRing(Integers(900))
R

Multivariate Polynomial Ring in a, b, c, d, e, f, g, h, i over Ring of integers modulo 900

In [45]:
# F = sage.rings.ideal.Katsura(R,8)
F =  R.ideal(a + a*b + b*c + c*d + d*e + e*f -   g, b + a*c + b*d + c*e + d*f - 2*g, c + a*d + b*e + c*f - 3*g, d + a*e + b*f - 4*g, e + a*f - 5*g, f - 6*g, a + b + c + d + e + f + 1)
F

Ideal (a*b + b*c + c*d + d*e + e*f + a + 899*g, a*c + b*d + c*e + d*f + b + 898*g, a*d + b*e + c*f + c + 897*g, a*e + b*f + d + 896*g, a*f + e + 895*g, f + 894*g, a + b + c + d + e + f + 1) of Multivariate Polynomial Ring in a, b, c, d, e, f, g, h, i over Ring of integers modulo 900

In [47]:
%time GB_PIR_F4(F)

CPU times: user 251 ms, sys: 7.94 ms, total: 259 ms
Wall time: 293 ms


[25*g^5 + 725*g^4 + 100*d^3 + 675*d^2*e + 225*d*e*g + 725*d*g^2 + 800*e*g^2 + 225*g^3 + 375*c^2 + 500*d^2 + 725*c*e + 500*d*e + 375*c*g + 700*e*g + 800*g^2 + 650*b + 400*d + 325*e + 775*g + 425, c^3 + 748*d^3 + 572*d^2*e + 72*d*e^2 + 324*e^3 + 425*d^2*g + 468*e^2*g + 225*d*g^2 + 491*e*g^2 + 630*g^3 + 756*c*d + 603*d^2 + 581*c*e + 712*d*e + 756*e^2 + 422*c*g + 325*d*g + 468*e*g + 582*g^2 + 897*b + 447*c + 72*d + 536*e + 236*g + 899, c^2*g + 448*d*e*g + 324*e^2*g + 288*d*g^2 + e*g^2 + 378*g^3 + 300*c^2 + 800*c*d + 200*d^2 + 283*c*e + 433*d*e + 108*e^2 + 786*c*g + 136*d*g + 684*e*g + 342*g^2 + 750*b + 875*c + 571*d + 216*e + 610*g, c*g^2 + 451*d*g^2 + 376*e*g^2 + 831*g^3 + 25*d^2 + 700*c*e + 216*e^2 + 875*c*g + 279*d*g + 224*e*g + 140*g^2 + 325*e + 25*g, b^2 + 448*c*d + 899*d^2 + 223*c*e + 896*d*e + 648*e^2 + 663*c*g + 576*d*g + 764*e*g + 642*g^2 + 2*b + 2*c + 452*d + 377*e + 788*g + 1, b*c + c^2 + 452*c*d + d^2 + 676*c*e + 452*d*e + 576*e^2 + 231*c*g + 756*d*g + 612*e*g + 361*g^2 + 899*b

In [71]:
%time GB_PIR_F4(F) # eco 9

CPU times: user 8min 14s, sys: 716 ms, total: 8min 15s
Wall time: 8min 15s


Polynomial Sequence with 66 Polynomials in 9 Variables

# [Top](#Top)